In [2]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
import scipy as sp
import scipy.io
import scipy.stats
import os
import numpy as np
import pandas as pd
import glob
import csv
import random as rand
from tqdm import tnrange, tqdm_notebook

import cmocean
import matplotlib.patches as patches
from matplotlib import gridspec

from collections import Iterable
import matplotlib.pylab as mpl
import random as rand
from ipywidgets import *
import colorlover as cl
import sys
sys.path.append(os.getcwd()+'/../')
from utils import utils, zscores


from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize


C:\Users\Eric\AppData\Local\Temp/ipykernel_6872/1158576645.py:18: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [3]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 24}

mpl.rc('font', **font)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
mpl.rc('axes', labelsize=20)

bin_size = 0.025

In [24]:
DATA_DIR = os.path.abspath('../data')
##### uncomment below if the processed data is already saved
log_df_1cyc = pd.read_hdf(os.path.join(DATA_DIR, 'log_df_processed_02-28-2019.h5'), 'fixed').reset_index(drop = True)
unit_key_df_1cyc = pd.read_hdf(os.path.join(DATA_DIR, 'unit_key_df_processed_02-28-2019.h5'), 'fixed')

In [26]:
z_score_bl_df = log_df_1cyc.copy()

In [27]:
baseline_df = pd.DataFrame(-np.column_stack((z_score_bl_df['stim_onset'], np.repeat(0.5, len(z_score_bl_df['stim_onset'])))),
                  columns = ['baseline_min', 'baseline_max'])

In [28]:
len(np.repeat(0.5, len(z_score_bl_df['stim_onset'])))

453926

In [29]:
np.column_stack((z_score_bl_df['stim_onset'], np.repeat(0.5, len(z_score_bl_df['stim_onset'])))).shape

(453926, 2)

In [30]:
baseline_df = pd.DataFrame(-np.column_stack((z_score_bl_df['stim_onset'], np.repeat(0.5, len(z_score_bl_df['stim_onset'])))),
                  columns = ['baseline_min', 'baseline_max'])

In [31]:
baseline_df

,baseline_min,baseline_max
0,-5.231200,-0.5
1,-3.007500,-0.5
2,-1.640900,-0.5
3,-7.220600,-0.5
4,-1.426950,-0.5
...,...,...
453921,-4.896933,-0.5
453922,-2.192833,-0.5
453923,-5.315067,-0.5
453924,-3.606233,-0.5


In [32]:
z_score_bl_df = log_df_1cyc.copy()

# ITI is added on to the beginning of each trial therefore stim-onset time is a direct measure of ITI length.
# will restrict baseline periods to 0.5 s before stim onset - periods will be [-stimOnset to -0.5]
baseline_df = pd.DataFrame(-np.column_stack((z_score_bl_df['stim_onset'], np.repeat(0.5, len(z_score_bl_df['stim_onset'])))),
                  columns = ['baseline_min', 'baseline_max'])
z_score_bl_df = pd.concat((z_score_bl_df, baseline_df), axis=1)

# remove all trials with dropped stim-onsets that are due to premature licking
z_score_bl_df['baseline_min'] = z_score_bl_df['baseline_min'].replace(0, np.nan)
z_score_bl_df = z_score_bl_df.dropna(subset = ['baseline_min'])

# remove all trials with short ITIs and truncate ITIs of trials with ITIs longer than 3.5s
z_score_bl_df = z_score_bl_df[z_score_bl_df['baseline_min'] <= -3.5]  
z_score_bl_df.loc[:, 'baseline_min'] = -3.5


edges = np.arange(-3.5, -1, bin_size)
z_score_bl_df['baseline_spike_count'] = [np.histogram(trial_spikes, edges)[0] 
                                         for trial_spikes in z_score_bl_df['spike_times(stim_aligned)']]

In [34]:
unit_key_df_1cyc['FR_mean'] = np.nan
unit_key_df_1cyc['FR_std'] = np.nan
import warnings

for num in tnrange(len(unit_key_df_1cyc.index)):
    unit = unit_key_df_1cyc.index[num]
    unit_rows = z_score_bl_df[(z_score_bl_df['mouse_name'] == unit_key_df_1cyc.loc[unit, 'mouse_name']) &
                              (z_score_bl_df['date'] == unit_key_df_1cyc.loc[unit,'date']) &
                              (z_score_bl_df['cluster_name'] == unit_key_df_1cyc.loc[unit,'cluster_name'])]
    mouse_name = unit_rows['mouse_name'].iloc[0]
    date = unit_rows['date'].iloc[0]
    cluster_name = unit_rows['cluster_name'].iloc[0]
    
    ## since there are 8 trial types (short/long x touchStim/visStim x touchBlock/visBlock) will use random sample of
    ## 1/8 of all trials to calculate mean and std of baseline firing rate for each unit
    baseline_trial_inds = rand.sample(list(unit_rows.index), int(np.max(unit_rows['trial_num'])/8))
    bin_means = np.mean(np.stack(unit_rows.loc[baseline_trial_inds, 
                                               'baseline_spike_count'].values, axis = 0), axis=0)/bin_size

    unit_key_df_1cyc.loc[unit,'FR_mean'] = np.mean(bin_means)
    unit_key_df_1cyc.loc[unit,'FR_std'] = np.std(bin_means)

unit_key_df_1cyc = unit_key_df_1cyc[unit_key_df_1cyc['FR_std'] != 0].reset_index(drop = True)

C:\Users\Eric\AppData\Local\Temp/ipykernel_6872/1840706730.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for num in tnrange(len(unit_key_df_1cyc.index)):


  0%|          | 0/1539 [00:00<?, ?it/s]

In [35]:
log_df_1cyc['mouse_name'].unique()

array(['EF0074', 'EF0076', 'EF0077', 'EF0079', 'EF0081', 'EF0084',
       'EF0088', 'EF0089', 'EF0114'], dtype=object)

In [36]:
log_df_1cyc['mouse_name'].unique()

array(['EF0074', 'EF0076', 'EF0077', 'EF0079', 'EF0081', 'EF0084',
       'EF0088', 'EF0089', 'EF0114'], dtype=object)

In [37]:
window = [-1, 3]
edges = np.arange(window[0], window[1], bin_size)

log_df_1cyc['spike_counts(stim_aligned)'] = log_df_1cyc['spike_times(stim_aligned)'].apply(lambda x: np.histogram(x,edges)[0]/bin_size)
log_df_1cyc.loc['spike_counts(stim_aligned)'] = log_df_1cyc.loc['spike_times(stim_aligned)'].apply(lambda x: x/bin_size)

# log_df_1cyc['spike_counts(lick_aligned)'] = log_df_1cyc['spike_times(lick_aligned)'].apply(lambda x: np.histogram(x,edges)[0]/bin_size)
# log_df_1cyc.loc['spike_counts(lick_aligned)'] = log_df_1cyc.loc['spike_times(lick_aligned)'].apply(lambda x: x/bin_size)

conds = ['Touch Stim Lick', 'Touch Stim No Lick', 'Short Touch Stim Lick', 'Short Touch Stim No Lick',
         'Visual Stim Lick', 'Visual Stim No Lick', 'Short Visual Stim Lick', 'Short Visual Stim No Lick']

z_conds =['Touch Stim Lick(z_score)', 'Touch Stim No Lick(z_score)', 'Short Touch Stim Lick(z_score)',
          'Short Touch Stim No Lick(z_score)','Visual Stim Lick(z_score)', 'Visual Stim No Lick(z_score)',
          'Short Visual Stim Lick(z_score)', 'Short Visual Stim No Lick(z_score)']

for c in range(8):
    unit_key_df_1cyc[conds[c]] = np.nan
    unit_key_df_1cyc[conds[c]] = unit_key_df_1cyc[conds[c]].astype(object)
    unit_key_df_1cyc[z_conds[c]] = np.nan
    unit_key_df_1cyc[z_conds[c]] = unit_key_df_1cyc[z_conds[c]].astype(object)
    

KeyError: 'spike_times(stim_aligned)'

In [ ]:
for i in tnrange(len(unit_key_df_1cyc.index)):
    unit = unit_key_df_1cyc.index[i]

    unit_rows = log_df_1cyc[((log_df_1cyc['mouse_name'] == unit_key_df_1cyc.loc[unit, 'mouse_name']) &
                        (log_df_1cyc['date'] == unit_key_df_1cyc.loc[unit,'date']) &
                        (log_df_1cyc['cluster_name'] == unit_key_df_1cyc.loc[unit,'cluster_name']))]

    ts_lick = unit_rows[(unit_rows['block_type'] == 'Whisker') & (unit_rows['correct'] == 1) &
                       ((unit_rows['trial_type'] == 'Stim_Som_NoCue') | (unit_rows['trial_type'] == 'Stim_Som'))]
    ts_no_lick = unit_rows[(unit_rows['response'] == 0) &
                      ((unit_rows['trial_type'] == 'Stim_Som_NoCue')| (unit_rows['trial_type'] == 'Stim_Som'))]
    short_ts_lick = unit_rows[(unit_rows['block_type'] == 'Whisker') & (unit_rows['correct'] == 1) &
                       ((unit_rows['trial_type'] == '1CycStim_Som_NoCue') | (unit_rows['trial_type'] == '1CycStim_Som'))]
    short_ts_no_lick = unit_rows[(unit_rows['response'] == 0) &
                      ((unit_rows['trial_type'] == '1CycStim_Som_NoCue')| (unit_rows['trial_type'] == '1CycStim_Som'))]
   
    vs_lick = unit_rows[(unit_rows['block_type'] == 'Visual') &(unit_rows['correct'] == 1) & 
                      ((unit_rows['trial_type'] == 'Stim_Vis_NoCue')| (unit_rows['trial_type'] == 'Stim_Vis'))]
    vs_no_lick = unit_rows[(unit_rows['response'] == 0) &
                      ((unit_rows['trial_type'] == 'Stim_Vis_NoCue')| (unit_rows['trial_type'] == 'Stim_Vis'))]
    short_vs_lick = unit_rows[(unit_rows['block_type'] == 'Visual') &(unit_rows['correct'] == 1) & 
                      ((unit_rows['trial_type'] == '1CycStim_Vis_NoCue')| (unit_rows['trial_type'] == '1CycStim_Vis'))]
    short_vs_no_lick = unit_rows[(unit_rows['response'] == 0) &
                      ((unit_rows['trial_type'] == '1CycStim_Vis_NoCue')| (unit_rows['trial_type'] == '1CycStim_Vis'))]
    
    trial_types = [ts_lick, ts_no_lick, short_ts_lick,short_ts_no_lick,vs_lick,
                   vs_no_lick, short_vs_lick, short_vs_no_lick]
    
    bin_means_all = []
    
    for j in range(8):
        if trial_types[j].shape[0] == 0:
            continue
        else:
            bin_means = np.mean(np.stack(trial_types[j]['spike_counts(stim_aligned)'].values, axis = 0), axis=0)
            baseline_mean = unit_key_df_1cyc.loc[unit, 'FR_mean']
            baseline_std = unit_key_df_1cyc.loc[unit, 'FR_std']

            unit_key_df_1cyc.at[unit,conds[j]] = bin_means

            unit_key_df_1cyc.at[unit,z_conds[j]] = (bin_means - baseline_mean)/baseline_std
   

In [ ]:
unit_key_df_1cyc

In [ ]:
unit_key_df_1cyc[trial_types].dropna()

In [ ]:
maps

In [ ]:
data['Short Visual Stim Lick(z_score)']

In [ ]:
mpl.close('all')
sub_map = unit_key_df_1cyc
colors = cmocean.cm.thermal
columns = ['Touch Stim Lick(z_score)', 'Short Touch Stim Lick(z_score)',
           'Touch Stim No Lick(z_score)', 'Short Touch Stim No Lick(z_score)',
           'Visual Stim Lick(z_score)', 'Short Visual Stim Lick(z_score)',
           'Visual Stim No Lick(z_score)', 'Short Visual Stim No Lick(z_score)']
data = {}
for z_map in columns:
    data[z_map] = pd.DataFrame(np.stack(sub_map[z_map].dropna().values))

# py.iplot(fig, filename= 'Hit_CR_Miss')


fig5c_1 = mpl.figure(figsize = (8,7))

gs1 = gridspec.GridSpec(1,1)
gs2 = gridspec.GridSpec(1,1)

gs1.update(bottom=0.1, top=0.68, left = 0.15, right = 0.9)
gs2.update(bottom = 0.69, top=0.87, left = 0.15, right = 0.9)

ax1 = mpl.subplot(gs1[0, 0])
ax2 = mpl.subplot(gs2[0, 0], sharex =ax1)

ax2.axis('off')
ax2.add_patch(patches.Rectangle((0,1.6), 0.15, 1, facecolor = 'k'))
ax2.add_patch(patches.Rectangle((0,.4), 0.05, 1, facecolor = 'k'))
ax2.set_ylim(0,4.7)


maps = [data[col] for col in columns]
m_psth = []
sem_psth = []

for z_map in maps:
    m_psth.append(z_map.mean(axis = 0))
    sem_psth.append(scipy.stats.sem(z_map, axis = 0))

colors = ['#7a0177', '#c51b8a', [0.2,.2,0.2,0.2], [0.4,0.4,0.4,0.4],  'C1','#feb24c', '#31a354', '#006837']

xvals = np.arange(-1,3,bin_size)
for col in range(len(columns)):
    ax1.plot(xvals[:-1],m_psth[col], color = colors[col])
    ax1.fill_between(xvals[0:-1], m_psth[col]-sem_psth[col], m_psth[col]+sem_psth[col],
                     alpha=0.5,color = colors[col])
  
  
# ax1.text(.8, .8, "Hit" , size=18, transform=ax1.transAxes, color = colors[0])
ax1.text(.025, .96, "Long tactile lick" , size=15, transform=ax1.transAxes, color = colors[0], alpha = 1)
ax1.text(.025, .9, "Short tactile lick" , size=15, transform=ax1.transAxes, color = colors[1], alpha = 1)
ax1.text(.025, .84, "Long visual lick" , size=15, transform=ax1.transAxes, color = colors[4], alpha = 1)
ax1.text(.025, .78, "Short visual lick" , size=15, transform=ax1.transAxes, color = colors[5], alpha = 1)
ax1.text(.525, .96, "Long tactile no lick" , size=15, transform=ax1.transAxes, color = colors[2] , alpha = 1)
ax1.text(.525, .9, "Short tactile no lick" , size=15, transform=ax1.transAxes, color = colors[3] , alpha = 1)
ax1.text(.525, .84, "Long visual no lick" , size=15, transform=ax1.transAxes, color = colors[6], alpha = 1)
ax1.text(.525, .78, "Short visual no lick" , size=15, transform=ax1.transAxes, color = colors[7] , alpha = 1)
ax1.set_xlabel('Time from stim onset (s)')
ax1.set_ylabel('Mean Z-score')
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_xlim(-.5,1)
ax1.set_ylim(-.5,2.1)


fig5c_1

In [ ]:
fig5c_1.savefig('fig3c_1.pdf', format='pdf', dpi=900)

In [ ]:
sorted_unit_key_df_1cyc.columns

In [ ]:
unit_key_df.columns

In [ ]:
unit_key_df_1cyc['abs_touch_RT_diff'] = (np.abs(unit_key_df_1cyc['RT_median_sTLR'] - unit_key_df_1cyc['RT_median_TLR']))
sorted_unit_key_df_1cyc = unit_key_df_1cyc.sort_values(['abs_touch_RT_diff'])

In [ ]:
mpl.close('all')
import matplotlib.patches as patches
from matplotlib import gridspec

fig4c_2 = mpl.figure(figsize = (15,14))
gs1 = gridspec.GridSpec(1,3)
gs2 = gridspec.GridSpec(1,1)
gs1.update(bottom=0.50, top=0.86, hspace=0.15, wspace=0.3)
gs2.update(left = 0.33, right = 0.66, bottom=0.1, top=0.38, wspace=0.05)


ax1 = mpl.subplot(gs1[0, 0])
ax2 = mpl.subplot(gs1[0, 2])
ax3 = mpl.subplot(gs1[0, 1])
ax6 = mpl.subplot(gs2[:])


for unit in range(sorted_unit_key_df_1cyc.shape[0]):
    ax1.add_patch(patches.Rectangle((0, unit), sorted_unit_key_df_1cyc.loc[unit, 'RT_median_VLL']*-1, 1, 
                                    edgecolor = 'w', facecolor = 'C1', linewidth = 0.1))
    ax1.add_patch(patches.Rectangle((0, unit), sorted_unit_key_df_1cyc.loc[unit, 'RT_median_TLR'], 1, 
                                    edgecolor = 'w', facecolor = 'C0' , linewidth = 0.1))
    
    ax3.add_patch(patches.Rectangle((0, unit), sorted_unit_key_df_1cyc.loc[unit, 'Vis_late_onset']*-1, 1, 
                                   edgecolor = 'w', facecolor ='C1', linewidth = 0.1))
    ax3.add_patch(patches.Rectangle((0, unit), sorted_unit_key_df_1cyc_1cyc.loc[unit, 'Tac_late_onset'], 1,
                                    edgecolor = 'w', facecolor ='C0', linewidth = 0.1))
    
    ax2.add_patch(patches.Rectangle((0, unit), sorted_unit_key_df_1cyc.loc[unit, 'median_diff'], 1, 
                                    edgecolor = 'k', facecolor ='k'))
    ax2.add_patch(patches.Rectangle((0, unit), sorted_unit_key_df_1cyc.loc[unit, 'late_onset_diff'], 1, 
                                    edgecolor = 'r', facecolor ='r', alpha = 0.8))
ax6.scatter(bimodal_lick_units['median_diff'], bimodal_lick_units['late_onset_diff'], color = 'C6')
ax6.plot(x, diff_fitline_values, 'k')

ax1.set_xlim(-1.5,1.5)
ax1.set_ylim(0,sorted_unit_key_df_1cyc.shape[0])
ax2.set_xlim(-1.5,1.5)
ax2.set_ylim(0,sorted_unit_key_df_1cyc.shape[0])
ax3.set_xlim(-1.5,1.5)
ax3.set_ylim(0,sorted_unit_key_df_1cyc.shape[0])
ax6.set_xlim(-1,1)
ax6.set_ylim(-1,1)

ax2.set_xlabel('Time (s)')
ax2.text(.6, .4, "Median reaction\ntime difference" , size=20, transform=ax2.transAxes, color = 'k')
ax2.text(.6, .25, "Late-activity\nonset difference" , size=20, transform=ax2.transAxes, color = 'r')

ax3.set_ylabel('Unit')
ax6.set_ylabel('Time (s)')

ax6.set_xlabel('Median reaction time\ndifference (s)')
ax6.set_ylabel('Late-activity\nonset difference')
ax6.text(.8, .65, "r$^2$ = "+str(np.around(vis_r**2, 2)) , size=20, transform=ax6.transAxes, color = 'k')


for ax in [ax1,ax2, ax3, ax6]:
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.spines['top'].set_visible(False)

    
ax1.spines['bottom'].set_visible(False)
ax1.get_xaxis().set_visible(False)

    
ax3.spines['bottom'].set_visible(False)
ax3.get_xaxis().set_visible(False)


ax1.set_ylabel('Unit')
ax2.set_ylabel('Unit')


ax4 = fig4c_2.add_axes([0.233, 0.48, 0.072, 0.1])
ax4.patch.set_alpha(0)
ax4.spines['left'].set_visible(False)
ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)
ax4.get_yaxis().set_visible(False)
ax4.set_xlabel('Median RT\ntactile trials (s)')

ax5 = fig4c_2.add_axes([0.161, 0.48, 0.072, 0.40])
ax5.invert_xaxis()

ax5.set_xlabel('Median RT\nvisual trials (s)', labelpad = 20)
ax5.patch.set_alpha(0)
ax5.spines['left'].set_visible(False)
ax5.spines['bottom'].set_visible(False)
ax5.spines['right'].set_visible(False)
ax5.get_yaxis().set_visible(False)
ax5.xaxis.set_ticks_position('top')
ax5.xaxis.set_label_position('top')


ax7 = fig4c_2.add_axes([0.513, 0.48, 0.072, 0.1])
ax7.patch.set_alpha(0)
ax7.spines['left'].set_visible(False)
ax7.spines['top'].set_visible(False)
ax7.spines['right'].set_visible(False)
ax7.get_yaxis().set_visible(False)
ax7.set_xlabel('Onset of late activity\ntactile trials (s)')

ax8 = fig4c_2.add_axes([0.441, 0.48, 0.072, 0.40])
ax8.invert_xaxis()

ax8.set_xlabel('Onset of late activity\nvisual trials (s)', labelpad = 20)
ax8.patch.set_alpha(0)
ax8.spines['left'].set_visible(False)
ax8.spines['bottom'].set_visible(False)
ax8.spines['right'].set_visible(False)
ax8.get_yaxis().set_visible(False)
ax8.xaxis.set_ticks_position('top')
ax8.xaxis.set_label_position('top')


fig4c_2